In [ ]:
# default_exp gui.main

# ExPoCo GUI

> The main expoco app.

TODO: abstract out win32 bits

In [17]:
#export
from expoco.core import *
from expoco.ml.data import *
from expoco.ml.model import *
from expoco.gui.capture_command import *
import numpy as np
from pathlib import Path
import cv2, time, math, json
import win32api, win32con
from PIL import Image

import mediapipe as mp
mp_face_mesh = mp.solutions.face_mesh

from dataclasses import dataclass
from collections import namedtuple

# Model

In [2]:
#export
# model_path = Path('data/viseme_dataset_20211116_113131/processed_20211116_131807/model_20211116_132150')
# relative_landmark_id=FaceLandmarks.tip_of_nose
model_path = Path('data/viseme_dataset_20211116_113131/processed_20211117_200746/model_20211117_201151')
relative_landmark_id=None
model = load_tabular_model(model_path)

In [3]:
#export
with open(model_path.parent/'metadata.json') as f:
    metadata = json.load(f)
cont_names = metadata['cont_names']
y_name = metadata['y_name']
stats = np.load(model_path.parent/'stats.npz')

# other ... todo

In [4]:
#export
screen_width, screen_height = win32api.GetSystemMetrics(0), win32api.GetSystemMetrics(1)
print('screen_width, screen_height', screen_width, screen_height)

screen_width, screen_height 1280 720


In [5]:
#export
@dataclass
class FacePoint:
    x: int
    y: int
    def __call__(self): return self.x, self.y
    def __repr__(self): return f'({self.x},{self.y})'
    
@dataclass
class FacePoints:
    tip_of_nose: FacePoint
    up: FacePoint
    down: FacePoint
    left: FacePoint
    right: FacePoint
    face_point_ids = [1, 5, 2, 218, 438]

In [6]:
#export
class FacePointHelper:
    def __init__(self, image_width, image_height, screen_width, screen_height): # TODO: remove screen_width, screen_height
        self.image_width, self.image_height = image_width, image_height
        self.screen_width, self.screen_height = screen_width, screen_height
        self.face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1)
        
    def process(self, image):
        self.results = self.face_mesh.process(image) # cv2.cvtColor(image, cv2.COLOR_BGR2RGB) already done
        return self.results
        
    def face_points(self, pixel_coordinates=True):
        fn = self._landmark_to_pixel_coordinates if pixel_coordinates else self._landmark_to_x_y
        return FacePoints(
            *[FacePoint(*fn(i)) for i in FacePoints.face_point_ids])
        
    def face_orientation(self, calibration): # TODO: better name, this is face angle / where it's pointing to / face|facial orientation
        points = self.face_points(False)
        l = points.tip_of_nose.x - points.left.x
        r = points.right.x - points.tip_of_nose.x
        u = points.tip_of_nose.y - points.up.y
        d = points.down.y - points.tip_of_nose.y
        pan = (l - r) - calibration[0] 
        tilt = (u - d) - calibration[1] # - 0.018 # TODO: calibrate
        return pan, tilt, 0.0

    def _landmark(self, i):
        return self.results.multi_face_landmarks[0].landmark[i] # [0] is OK as we're running with max_num_faces=1
        
    def _is_valid_normalized_value(self, value):
        return (value > 0 or math.isclose(0, value)) and (value < 1 or math.isclose(1, value))
    
    def _normalized_x_to_pixel(self, value):
        return math.floor(value * self.image_width)
    
    def _normalized_y_to_pixel(self, value):
        return math.floor(value * self.image_height)
    
    def _landmark_to_x_y(self, landmark):
        if isinstance(landmark, int):
            landmark = self._landmark(landmark)
        if not (self._is_valid_normalized_value(landmark.x) and self._is_valid_normalized_value(landmark.y)):
            print(f'WARNING: {landmark.x} or {landmark.y} is not a valid normalized value')
        return landmark.x, landmark.y
    
    def _landmark_to_pixel_coordinates(self, landmark):
        x, y = self._landmark_to_x_y(landmark)
        return self._normalized_x_to_pixel(x), self._normalized_y_to_pixel(y)

In [7]:
#export
def calibrate():
    win32api.SetCursorPos([screen_width//2, screen_height//2])
    time.sleep(.5)
    return face_point_helper.face_orientation([0.0,0.0,0.0])

calibration = (0.0, 0.02087956666946411, 0.0)

def _r(l): return [round(i, 5) for i in l]

def move(yaw, pitch):
    pointer_x_px, pointer_y_px = list(win32api.GetCursorPos())
    pointer_x_px -= screen_width//2
    pointer_x_px /= 6.5e4 # 4.5e4 # was 5.5e4 # TODO: calibrate/pref
    pointer_speed = 250 # TODO: user pref
    x_move = math.floor(((yaw - pointer_x_px) * pointer_speed)**3) 
    pointer_y_px -= screen_height//2
    pointer_y_px /= 6.5e4 # 4.5e4 # was 5.5e4 # TODO: calibrate
    y_move = math.floor(((pitch - pointer_y_px) * pointer_speed)**3) 
    x, y = win32api.GetCursorPos()
    win32api.SetCursorPos([x + x_move, y + y_move])

In [37]:
#export
class StopExpocoException(Exception):
    pass
def stop_expoco():
    raise StopExpocoException()

In [38]:
#export
Command = namedtuple('Command', ['key', 'label', 'fn'])
commands = [
    Command('q', 'Quit', stop_expoco),
    Command('l', 'Left click', lambda: pointer_left_click(*get_pointer_position())),
    Command('r', 'Right click', lambda: pointer_right_click(*get_pointer_position())),
    Command('semicolon', 'Enter / Return', lambda: win32api.keybd_event(win32con.VK_RETURN, 0)),
    Command('e', 'End', lambda: win32api.keybd_event(win32con.VK_END, 0)),
    Command('h', 'Home', lambda: win32api.keybd_event(win32con.VK_HOME, 0)),
    Command('d', 'Delete', lambda: win32api.keybd_event(win32con.VK_DELETE, 0)),
    Command('b', 'Backspace', lambda: win32api.keybd_event(win32con.VK_BACK, 0)),
    Command('as', 'Test seq', lambda: print('"as" sequence hit')),
]
command_map = {c.key: c for c in commands}

In [42]:
#export
def run_main():
    try: video_capture.release() # TODO: del
    except: pass
    try:
        video_capture = cv2.VideoCapture(0) 
        face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1)
        for vk in [win32con.VK_ESCAPE, ord('C')]: win32api.GetAsyncKeyState(vk)
        retval, image = video_capture.read()
        face_point_helper = FacePointHelper(*image.shape[:2], screen_width, screen_height)
        hist_len = 15
        x_hist, y_hist, hist_idx = np.zeros(hist_len), np.zeros(hist_len), 0
        while True:
            retval, image = video_capture.read()
            image = cv2.flip(image, 1)
            results = face_point_helper.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            face_orientation = face_point_helper.face_orientation(calibration)
            move(face_orientation[0], face_orientation[1])
        #     if win32api.GetAsyncKeyState(ord('C')):
        #         calibration = calibrate()
        #         print('calibration', calibration)

            #viseme start
            # TODO: read metadata.json and build dynamically
            data = inference_data_from_landmarks(
                    landmarks=results.multi_face_landmarks[0].landmark, 
                    landmark_ids=FaceLandmarks.pointer + FaceLandmarks.mouth,
                    relative_landmark_id=relative_landmark_id, 
                    coords=['x', 'y'], 
                    stats=stats)
            output = model(data)
            class_id = np.argmax(output)
            if class_id == 1:
                keys_pressed = capture_key_press([f"{c.key}: {c.label}" for c in commands], not pointer_in_left_half_of_screen())
                print('keys_pressed', keys_pressed)
                run_command(command_map, keys_pressed)
        #     class_label = viseme_config.get_class_label(class_id)
        #     print('class_id', class_id, output)
            #viseme end

    #         if win32api.GetAsyncKeyState(win32con.VK_ESCAPE): 
    #             break
            time.sleep(.05)
    except StopExpocoException:
        video_capture.release()
    finally:
        video_capture.release()

In [43]:
run_main()

keys_pressed ['q']


In [41]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 05a_ml_data.ipynb.
Converted 05b_ml_model.ipynb.
Converted 10a_mediapipe_face_mesh_identify_landmarks.ipynb.
Converted 10b_mediapipe_face_mesh_capture_session.ipynb.
Converted 10c_mediapipe_face_mesh_train_model.ipynb.
Converted 10d_test_np_model.ipynb.
Converted 20a_gui_capture_command.ipynb.
Converted 20a_gui_main.ipynb.
Converted 70_cli.ipynb.
Converted index.ipynb.
Converted project_lifecycle.ipynb.
